In [386]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from sklearn.metrics import mean_squared_error
import xgboost as xgb

df_data = pd.read_csv('hw4-trainingset-lr2965.csv')
df_list = pd.read_csv('FeaturesEdit.csv')
df_pred = pd.read_csv('hw4-testset-lr2965.csv')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (50,172,255,256,257,258,280,376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (50,255,256,257,258,260,268,280) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_namelist = df_list[df_list.Manipulation != 'Discard']


df_data = df_data[df_namelist.VarName]
df_pred = df_pred[df_namelist.VarName]

#print('Correlation: ', df_data['cntryid_e'].corr(df_data['job_performance']))

df_tlabel = df_data[['job_performance']].copy()



df_data = df_data.drop('job_performance', 1)
df_pred = df_pred.drop('job_performance', 1)
df_data = df_data.drop(['v71', 'isic2c', 'lng_home', 'reg_tl2'], 1)
df_pred = df_pred.drop(['v71', 'isic2c', 'lng_home', 'reg_tl2'], 1)


for attr in df_data:
  if df_data[attr].isna().sum() > 5000:
    df_data = df_data.drop(attr, 1)
    df_pred = df_pred.drop(attr, 1)





In [0]:
others = ['Netherlands', 'Czech Republic', 'Chile', 'Greece', 'Finland', 'Austria', 'Slovak Republic', 'Israel', 'Singapore', 'Sweden', 'New Zealand', 'Flanders (Belgium)', 'Denmark', 'Ireland', 'Norway', 'Lithuania', 'Slovenia', 'Estonia', 'Northern Ireland (UK)']

# Group less significant countries in Others
df_data['cntryid_e'] = df_data['cntryid_e'].replace(others, 'Other')
df_data['cntryid_e'] = df_data['cntryid_e'].replace('Canada (French)', 'Canada (English)')
df_pred['cntryid_e'] = df_pred['cntryid_e'].replace(others, 'Other')
df_pred['cntryid_e'] = df_pred['cntryid_e'].replace('Canada (French)', 'Canada (English)')




In [0]:
# Get Column name of all numeric columns
df_numeric = df_data.select_dtypes(include = 'number')
numeric = list(df_numeric.columns)


# Initialize Scaler 
Scaler = preprocessing.MinMaxScaler()

for col in numeric:
  df_data[col].fillna((int(round(df_data[col].mean()))), inplace=True)
  x = df_data[[col]].values
  x_scaled = Scaler.fit_transform(x)
  df_data[[col]] = pd.DataFrame(x_scaled)

  # The same for the test set 
  df_pred[col].fillna((int(round(df_pred[col].mean()))), inplace=True)
  xt = df_pred[[col]].values
  xt_scaled = Scaler.fit_transform(xt)
  df_pred[[col]] = pd.DataFrame(xt_scaled)

In [0]:
df_one_hot = df_list[df_list.Manipulation == 'onehot']
one_hot = df_one_hot['VarName'].values

intersection = list(set(one_hot) & set(df_data.columns))



df_train_processed = pd.get_dummies(df_data, columns = intersection, dummy_na = True)
df_test_processed = pd.get_dummies(df_pred, columns = intersection, dummy_na = True)



df_train_processed.drop(df_train_processed.tail(3).index,inplace=True)
df_tlabel.drop(df_tlabel.tail(3).index,inplace=True)

df_train_processed = df_train_processed.reindex(sorted(df_train_processed.columns), axis=1)
df_test_processed = df_test_processed.reindex(sorted(df_test_processed.columns), axis=1)

In [391]:
"""
mask_train = np.random.rand(len(df_processed)) < 0.8

df_train = df_train_processed[mask_train]
df_train_label = df_tlabel[mask_train]
df_valid = df_train_processed[~mask_train]
df_valid_label = df_tlabel[~mask_train]



dtrain = xgb.DMatrix(df_train, label=df_train_label, missing = -999.0)
dvalid = xgb.DMatrix(df_valid, label=df_valid_label, missing = -999.0)
#dtest = xgb.DMatrix(df_test_processed, missing = -999.0)

param = {'eval_metric': 'rmse', 'eta': 0.3}
num_round = 50
evallist = [(dvalid, 'valid'), (dtrain, 'train')]

bst = xgb.train(param, dtrain, num_round, evallist)

ypred = bst.predict(dvalid)

mse = mean_squared_error(df_valid_label, ypred)

print('The MSE is: ', mse)

#print('test predictions', ypred)


"""

dtrain = xgb.DMatrix(df_train_processed, label=df_tlabel, missing = -999.0)
dtest = xgb.DMatrix(df_test_processed, missing = -999.0)

param = {'eval_metric': 'rmse', 'eta': 0.3}
num_round = 414
evallist = [(dtrain, 'train')]

bst = xgb.train(param, dtrain, num_round, evallist)

ypred = bst.predict(dtest)

#mse = mean_squared_error(df_valid_label, ypred)

#print('The MSE is: ', mse)

print('test predictions', ypred)

np.savetxt("predictions.csv", ypred, delimiter=",")



[0]	train-rmse:2074.32
[1]	train-rmse:1471.57
[2]	train-rmse:1054.06
[3]	train-rmse:768.005
[4]	train-rmse:575.544
[5]	train-rmse:448.924
[6]	train-rmse:368.33
[7]	train-rmse:318.19
[8]	train-rmse:288.174
[9]	train-rmse:269.039
[10]	train-rmse:256.792
[11]	train-rmse:248.498
[12]	train-rmse:242.817
[13]	train-rmse:236.742
[14]	train-rmse:231.955
[15]	train-rmse:229.067
[16]	train-rmse:225.748
[17]	train-rmse:221.847
[18]	train-rmse:219.645
[19]	train-rmse:215.343
[20]	train-rmse:213.033
[21]	train-rmse:209.134
[22]	train-rmse:207.313
[23]	train-rmse:205.244
[24]	train-rmse:203.241
[25]	train-rmse:200.514
[26]	train-rmse:197.131
[27]	train-rmse:195.057
[28]	train-rmse:192.093
[29]	train-rmse:189.172
[30]	train-rmse:188.061
[31]	train-rmse:186.882
[32]	train-rmse:185.962
[33]	train-rmse:183.592
[34]	train-rmse:181.485
[35]	train-rmse:178.91
[36]	train-rmse:177.162
[37]	train-rmse:175.575
[38]	train-rmse:174.441
[39]	train-rmse:173.67
[40]	train-rmse:172.45
[41]	train-rmse:170.512
[42]	tr